# WiseWheels - what is behind it
Below I will record the data processing, feature engineering, and model selection for the WiseWheels project. The raw trip data (12 .csv files for 2016) is already preprocessed and pickled as a .pkl file. Other related files such as stations, weather, Yelp (in csv form) are saved in other location, and loaded on the fly.
### Premeable

In [ ]:
%matplotlib inline
import os, sys, time
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
## my function library
import function_lib as fun

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (14, 6)

data_year = '2016'
## get the path two levels up
data_path = '/'.join(os.getcwd().split('/')[:-2]) + '/python projects/CitiBike/'

### Read the "mega data" in!

In [ ]:
print('Reading the raw pickled data...')
trips = pd.read_pickle(data_path + 'citibike_'+ data_year +'.pkl')
print('Reading the raw pickled data done!')
## only keep the Subscriber
trips = trips[trips['usertype'] == 'Subscriber']
## let's build the training set
features = ['starttime', 'start station id',
           'birth year', 'gender', 'date', 'hour', 'weekday']
target = ['tripduration', 'end station id']
trips_lite = trips[features + target]
trips_lite['day of week'] = trips_lite['date'].dt.dayofweek
trips_lite['age'] = 2016 - trips_lite['birth year']
trips_lite.head()

## get the end neighborhood
df_stations = pd.read_csv('./data/NYC_bike_stations_v7.csv')
trips_lite = pd.merge(trips_lite, df_stations[['id', 'neighborhood']],
                     left_on = 'end station id', right_on = 'id', how = 'left')
trips_lite = trips_lite.rename(columns = {'neighborhood': 'end neighborhood'})

## merage with start station information
trips_lite = pd.merge(trips_lite, df_stations[['id', 'neighborhood',
            'shortest_dist','total_slots','closest_income',
            'closest_pop','nearest_3','nearest_5']], 
         left_on = 'start station id', right_on = 'id', how = 'left')
trips_lite = trips_lite.rename(columns = {'neighborhood': 'start neighborhood'})

## add the info from yelp
df_yelp = pd.read_csv('./data/station_attributes.csv')
print('Yelp data shape:', df_yelp.shape)
## remove the 'name'
df_yelp = df_yelp.drop(['name'], axis = 1)
print('Yelp data headers: ', df_yelp.columns)
trips_lite = pd.merge(trips_lite, df_yelp, left_on = 'start station id', right_on = 'id', 
     how = 'left')

## get the weather information, for each day
weather = pd.read_csv(data_path + 'weather_lite.csv', parse_dates = ['date'], infer_datetime_format = True)
trips_lite = trips_lite.merge(weather, on = 'date', how = 'left')
trips_lite.columns

## save 'some' of the data. If I try to save it all, it takes too much time
## somehow pickle throw me some error...
# trips_lite.sample(1000000).to_csv('./data/ready_to_train_2016_trips_lite.csv', index = False) 
print('data loaded.')
trips_lite.head()

# Let's start training!
### First let me take only part the 10 million+ samples

In [ ]:
## taking only 1%, which is about 100,000 samples
data = trips_lite.sample(int(1 / 100 * trips_lite.shape[0]))
print('Shape of the dataset is:', data.shape)
## let me check for the nan entris
print('Number of n/a entries are :', data.isnull().sum())
## drop the rows with any nan entry
data = data.dropna()
## delete the original file to free some space
del trips_lite

### Then let me load the station info, which will be used for new features, and merge

In [ ]:
cols = df_stations.columns
new_features = ['id']
for elem in cols:
    if elem.startswith('to ') or elem.startswith('freq to '):
        new_features.append(elem)
df_tmp = df_stations[new_features]
print('bike station df shape: {}'.format(df_tmp.shape))
# add these new features to the data
data = pd.merge(data, df_tmp, left_on = 'start station id', right_on = 'id', how = 'inner')
print('data df shape: {}'.format(data.shape))

### Feature selections

In [ ]:
features = ['start station id', 'hour', 'weekday', 'day of week',
            'age', 'gender', 
            'TAVG', 'PRCP', 'SNOW',
            'shortest_dist','total_slots','nearest_3','nearest_5',
            'closest_pop', 
            'closest_income', 
            # =========== following are from yelp ===========
            #'apartments', 'coffee', 'condominiums', 'convenience',
            #'elementaryschools', 'highschools', 'metrostations', 'pubs',
            #'restaurant', 'trainstations', 'artmuseums', 'auto', 'banks',
            #'bikerentals', 'businessfinancing', 'dryclean', 'gyms', 'hair',
            #'landmarks', 'laundromat', 'othersalons', 'paydayloans', 'preschools',
           ]
if 'id' in new_features:
    new_features.remove('id')  # remove the key column 
features.extend(new_features)  # whether to add the new station features: dists to all neighbors
target = ['end neighborhood']
data = data[features + target]
print('The shape of the data is:', data.shape)
data.sample(5).head()

### one_hoc encoding for the categorical features

In [ ]:
data = pd.get_dummies(data, columns=[
                                  'gender',
                                  'start station id',  'weekday', 'day of week'])
print('The shape of the data after one_hoc encoding is:', data.shape)

### Load the classifiers, and preprocess the data

In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

## choose what model to train
logistic_model = False
svm_model      = False
rf_model       = False
xgb_model      = False
nb_model       = False

seed = 42
train, test = train_test_split(data, test_size = 0.2, random_state = seed)
features = data.columns.tolist()
features.remove(*target)

# scaling the features
scaler = StandardScaler()
X_train, y_train = train[features], train[target].values.ravel()
X_test, y_test = test[features], test[target].values.ravel()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)  # apply same transformation to test data
print('number of training samples: {}'.format(len(X_train_scaled)))

### Logistic regression with SGD

In [ ]:
# ==================== Logistic classification ====================
if logistic_model:
    gridsearchLogistic = False  # whether to do grid search
    n_iter = 50  # number of iteration of SGD
    t_start = time.time()  # for timing purpose
    print('number of epochs:', n_iter)
    if not gridsearchLogistic:        
        ## to repeat few times and get the statistics
        n_iters = [50]*5
        train_scores = []
        test_scores = []
        for i, n_iter in enumerate(n_iters):
            print('working on {} of {}...'.format(i+1, len(n_iters)))
            single_clf = SGDClassifier(loss = 'log', n_iter=n_iter, n_jobs = -1, 
                                       verbose = 0, alpha = 0.0001,
                                       penalty = 'l2')
            single_clf.fit(X_train_scaled, y_train)
            train_score = single_clf.score(X_train_scaled, y_train)
            test_score  = single_clf.score(X_test_scaled, y_test)
            print('train scroe: {}'.format(train_score))
            print('test scroe: {}'.format(test_score))
            train_scores.append(train_score)
            test_scores.append(test_score)
            # fun.plot_conf_mat(clf=single_clf, X_test=X_test_scaled, y_test=y_test, N=5)
        print('mean and std for training are: {}, and {}'
             .format(np.array(train_scores).mean(), np.array(train_scores).std()))
        print('mean and std for test are: {}, and {}'
             .format(np.array(test_scores).mean(), np.array(test_scores).std()))
            
    else:  # let me do a grid search on some parameters
        parameters = {'alpha': 10.0**-np.arange(1,7)}
        single_clf = SGDClassifier(loss = 'log', n_iter=n_iter, n_jobs = -1, verbose = 0,
                                   penalty = 'l2')
        clf = GridSearchCV(single_clf, parameters, verbose = 1)
        clf.fit(X_train_scaled, y_train)
        print('best scroe is: {}'.format(clf.best_score_))
    print('Elapsed time is {} min'.format((time.time() - t_start)/60))

### Support Vector Machine with SGD

In [ ]:
# ==================== SVM ====================
from sklearn.svm import SVC
if svm_model:
    n_iter = 50  # number of iterations
    gridsearchSVM = False  # whether to do grid search
    t_start = time.time()  # for timing purpose
    print('number of epochs:', n_iter)
    if not gridsearchSVM:
        ## to repeat few times and get the statistics
        n_iters = [50]*5
        train_scores = []
        test_scores = []
        for i, n_iter in enumerate(n_iters):
            print('working on {} of {}...'.format(i+1, len(n_iters)))
            single_svm = SGDClassifier(loss = 'squared_hinge', n_iter=n_iter, n_jobs = 6, 
                                           verbose = 0, alpha = 0.0001, penalty = 'l1')
            single_svm.fit(X_train_scaled, y_train)
            train_score = single_svm.score(X_train_scaled, y_train)
            test_score  = single_svm.score(X_test_scaled, y_test)
            print('training scroe: {}'.format(train_score))
            print('test scroe: {}'.format(test_score))
            train_scores.append(train_score)
            test_scores.append(test_score)
        print('mean and std for training are: {}, and {}'
             .format(np.array(train_scores).mean(), np.array(train_scores).std()))
        print('mean and std for test are: {}, and {}'
             .format(np.array(test_scores).mean(), np.array(test_scores).std()))
    else:
        parameters = {'alpha': 10.0**-np.arange(1,7), 
                      'loss': ['hinge', 'squared_hinge'],
                      'penalty': ['l1', 'l2']
                     }
        single_svm = SGDClassifier(loss = 'squared_hinge', n_iter=n_iter, n_jobs = 4,
                                   verbose = 0)
        clf = GridSearchCV(single_svm, parameters, verbose = 1)
        clf.fit(X_train_scaled, y_train)
        print('best scroe is: {}'.format(clf.best_score_))
        print('best model is: {}'.format(clf.best_estimator_))
    print('Elapsed time is {} min'.format((time.time() - t_start)/60))

### Random Forest

In [ ]:
# ==================== Random Forest ====================
from sklearn.ensemble import RandomForestClassifier
if rf_model:
    t_start = time.time()
    gridsearchRF = False
    if not gridsearchRF:
        max_depth = 10
        n_iters = [50]*5
        train_scores = []
        test_scores = []
        for i, n_iter in enumerate(n_iters):
            print('working on {} of {}...'.format(i+1, len(n_iters)))
            rf_single = RandomForestClassifier(n_estimators=200,
                                                       max_depth=max_depth, n_jobs = 1,
                                                       verbose = True)
            rf_single.fit(X_train, y_train)
            train_score = rf_single.score(X_train, y_train)
            test_score  = rf_single.score(X_test, y_test)
            print('train scroe: {} \n'.format(train_score))
            print('test scroe: {} \n'.format(test_score))
            train_scores.append(rf_single.score(X_train, y_train))
            test_scores.append(rf_single.score(X_test, y_test))
        
        print('mean and std for training are: {}, and {}'
             .format(np.array(train_scores).mean(), np.array(train_scores).std()))
        print('mean and std for test are: {}, and {}'
             .format(np.array(test_scores).mean(), np.array(test_scores).std()))
        importances = [[c, i] for c, i in zip(
            data.columns, rf_single.feature_importances_)];
        importances = sorted(importances, key=lambda x: x[1], reverse = True);
        print('Feature importances:')
        for j in range(20):
            print(importances[j][0], importances[j][1])
        model_for_output = rf_single
    else:
        parameters = {'max_depth': [x for x in range(10, 41, 5)]}
        single_clf = RandomForestClassifier(n_estimators = 400, n_jobs = 1, verbose = False)
        clf = GridSearchCV(single_clf, parameters, verbose = 2)
        clf.fit(X_train, y_train)
        print('best model is : {}.\n'.format(clf.best_estimator_))
        print('best train scroe is: {} \n'.format(clf.best_score_))
        print('test score is: {} \n'.format(clf.best_estimator_.score(X_test, y_test)))
        importances = [[c, i] for c, i in zip(
            data.columns, clf.best_estimator_.feature_importances_)];
        importances = sorted(importances, key=lambda x: x[1], reverse = True);
        for j in range(10):
            print(importances[j][0], importances[j][1])
        # model_for_output = clf.best_estimator_
        
    X_sample = pd.Series(0, index = X_train.columns)
    model_name = 'Random_Forest.pkl'
    # check for the prediction
    print("\nLet's look at one test case:")
    idx = 1090
    debug_input_X, debug_y = X_test.iloc[idx,:], y_test[idx]      
    if not gridsearchRF:
        fun.get_proba(clf = model_for_output, X_scaled = debug_input_X, y_true = debug_y, N = 5)
        # pickle.dump([model_for_output, X_sample, debug_input_X, debug_y], open(model_name, 'wb'))
        ## plot confuction matrix
        # fun.plot_conf_mat(clf = model_for_output, X_test = X_test, y_test = y_test, N = 5)
        # fun.plot_conf_mat_bokeh(clf=rf_single, X_test=X_test, y_test=y_test)
    print('Elapsed time is {} min'.format((time.time() - t_start)/60))

### Gradient Boosted Trees (xgboost)

In [ ]:
# ========================== xgboost ==================================
if xgb_model:
    import xgboost as xgb
    ## the sklearn wrapper of xgboost seems to be problematic in this case:
    ## it simply doesn't stop! Therefore I am folding back to the most basic DMatrix
    ## in order to use DMatrix, the label needs to be numberic, I will convert it first
    ## good tutorial: http://ieva.rocks/2016/08/25/iris_dataset_and_xgboost_simple_tutorial/
    
    t_start = time.time()
    d_class_to_int = {}
    for i, elem in enumerate(np.unique(y_train)):
        d_class_to_int[elem] = i
    
    ## make the reverse dictionary for later use (if needed)
    d_int_to_class = {v: k for k, v in d_class_to_int.items()}
    
    y_train_xgb = np.vectorize(d_class_to_int.get)(y_train)
    y_test_xgb = np.vectorize(d_class_to_int.get)(y_test)
    
    ## make the train and test set
    dtrain = xgb.DMatrix(X_train, label=y_train_xgb)
    dtest = xgb.DMatrix(X_test, label=y_test_xgb)
    
    ## specify parameters via map
    param = {
         'max_depth': 50, 
         'eta': 0.3, # the training step for each iteration 
         'silent': 0, # logging mode, tell me sth
         'objective':'multi:softmax',  # multiclass problem
         'num_class': len(d_class_to_int),  # the number of classes that exist in this datset
         'nthread': 4, # the number of threads
        }
    num_round = 1 # the number of training iterations
    
    ## train!
    bst = xgb.train(param, dtrain, num_round)
    ## make prediction
    y_pred = bst.predict(dtest)
    ## to get the accuracy
    print('The time of xgboost is: {} minutes'.format((time.time() - t_start)/60))
    print('The train accuracy is: ', sum(bst.predict(dtrain) == y_train_xgb) / len(y_train_xgb))
    print('The test accuracy is: ', sum(y_pred == y_test_xgb) / len(y_pred))
    ## plot the feature importance
    #xgb.plot_importance(bst)
    #plt.show();

    ## to save the model, if needed      
    xbg_model_for_output = bst
    X_sample = pd.Series(0, index = X_train.columns)
    model_name = 'XGB.pkl'
    # pickle.dump([model_for_output, scaler, X_sample], open(model_name, 'wb'))
    print('Elapsed time is {} min'.format((time.time() - t_start)/60))

### Naive Bayes

In [ ]:
# ========================== Naive Bayes ==================================
if nb_model:
    from sklearn.naive_bayes import GaussianNB
    nb_clf = GaussianNB()
    nb_clf.fit(X_train_scaled, y_train)
    print('The test scroe for Naive Bayes is: {}'.format(nb_clf.score(X_test_scaled, y_test)))